## IMPORT LIBRARIES

In [4]:
# Import necessary libraries
from pyDOE2 import fracfact
from pyDOE2 import ff2n
import numpy as np
import pandas as pd
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.libqsturng import psturng
from statsmodels.stats.multitest import multipletests
from scipy.stats import t
import itertools
from itertools import combinations


## CREATE FACTORIAL DESIGN PLAN

In [5]:
# Function to generate factor levels DataFrame
def generate_factor_levels_df(factors, levels):
    """
    Generate a DataFrame with factor names and number of levels.

    Args:
    - factors (list): List of factor names.
    - levels (list): List containing the number of levels for each factor.

    Returns:
    - df (DataFrame): DataFrame representing factor names and levels.
    """
    if len(factors) != len(levels):
        raise ValueError("Number of factors and levels must be the same.")
    if not all(isinstance(level, int) and level > 0 for level in levels):
        raise ValueError("Levels must be positive integers.")
    
    data = {'Factor': factors, 'Levels': levels}
    df = pd.DataFrame(data)
    return df

# Function to generate a half fractional design matrix based on given factors
def half_fractional_design(factors_df):
    """
    Generates a half fractional design matrix based on the given factors.

    Args:
    - factors_df (DataFrame): DataFrame containing the factor names.

    Returns:
    - df (DataFrame): DataFrame containing the design matrix.
    - main_aliases (dict): Dictionary containing aliases for main effects.
    - two_way_interaction_aliases (dict): Dictionary containing aliases for two-way interactions.
    """
    factors = factors_df['Factor'].tolist()
    
    # Convert factors to standardized form
    standardized_factors = list('abcdefghijklmnopqrstuvwxyz')[:len(factors)]

    # Create the generator from the highest order interaction term
    generator = ''.join(standardized_factors[:-1])

    print("Design Generator", generator)

    # Correcting the input for fracfact
    fracfact_input = ' '.join(standardized_factors[:-1]) + ' ' + generator

    # Create the design using fracfact
    design = fracfact(fracfact_input)
    
    # Convert design matrix back to original factors
    df = pd.DataFrame(design[:, :len(factors)], columns=factors)  # Only select columns for main factors
    
    # Create the alias structure for main effects
    main_aliases = {}
    for i, factor in enumerate(factors):
        main_aliases[factor] = '*'.join([f for j, f in enumerate(factors) if (i != j)])
    
    # Create the alias structure for two-way interactions
    two_way_interaction_aliases = {}
    seen = set()
    for i in range(len(factors)):
        for j in range(i+1, len(factors)):
            interaction = factors[i] + '*' + factors[j]
            remaining_factors = [f for k, f in enumerate(factors) if (k != i and k != j)]
            alias = '*'.join(remaining_factors)
            if alias not in seen:
                two_way_interaction_aliases[interaction] = alias
                seen.add(interaction)
                seen.add(factors[j] + '*' + factors[i])
    
    return df, main_aliases, two_way_interaction_aliases

# Define the custom mapping function
def custom_mapping(value, levels):
    """
    Custom mapping function to transform values according to specific rules.

    Args:
    - value (int): Value representing factor level.
    - levels (int): Number of levels for the factor.

    Returns:
    - mapped_value (int): Transformed value.
    """
    if value < 0 or value >= levels:
        return value  # Value outside the defined levels, return as is
    
    if levels == 4:
        if value == 0:
            return -1
        elif value == 1:
            return -0.5
        elif value == 2:
            return 0.5
        elif value == 3:
            return 1
    elif levels == 3:
        if value == 0:
            return -1
        elif value == 1:
            return 0
        elif value == 2:
            return 1
    elif levels == 2:
        if value == 0:
            return -1
        elif value == 1:
            return 1

    # If the number of levels is not 2, 3, or 4, return the original value
    return value

In [6]:
# Prompt user for factors
while True:
    try:
        factors = input("Enter the factors separated by spaces: ").split()
        levels = [int(x) for x in input("Enter the number of levels for each factor separated by spaces: ").split()]
        factor_levels_df = generate_factor_levels_df(factors, levels)
        break
    except ValueError as e:
        print("Error:", e)

# Generate half fractional design based on factors
df, main_aliases, two_way_interactions_aliases = half_fractional_design(factor_levels_df)  # Change factors_df to factor_levels_df

# Prompt the user to enter the number of replicates
while True:
    try:
        num_replicates = int(input("Enter the number of replicates: "))
        if num_replicates <= 0:
            raise ValueError("Number of replicates must be a positive integer.")
        break
    except ValueError as e:
        print("Error:", e)

# Apply custom mapping to each value in the DataFrame
for column in df.columns:
    num_levels = factor_levels_df.loc[factor_levels_df['Factor'] == column, 'Levels'].iloc[0]
    df[column] = df[column].apply(lambda x: custom_mapping(x, num_levels))

# Duplicate the half fractional design DataFrame
df_duplicated = pd.concat([df] * num_replicates, ignore_index=True)

# Merge the two DataFrames
merged_df = pd.concat([factor_levels_df, pd.DataFrame(columns=['']), df_duplicated], axis=1)

# Add a blank column with heading 'Results' after the last factor
merged_df.insert(len(merged_df.columns), 'Results', '')

# Prompt the user to input the Excel file name
excel_file_name = input("Enter the Excel file name: ")

# Append '.xlsx' extension if not provided
if not excel_file_name.endswith('.xlsx'):
    excel_file_name += '.xlsx'

# Write the merged DataFrame to an Excel file
try:
    merged_df.to_excel(excel_file_name, index=False)
    print("Excel file saved successfully.")
except Exception as e:
    print("Error occurred while saving the Excel file:", e)

# Print the alias structures
print("Main Alias Structure:", main_aliases)
print("Two-way Interactions Alias Structure:", two_way_interactions_aliases)

Design Generator abc
Excel file saved successfully.
Main Alias Structure: {'A': 'B*C*D', 'B': 'A*C*D', 'C': 'A*B*D', 'D': 'A*B*C'}
Two-way Interactions Alias Structure: {'A*B': 'C*D', 'A*C': 'B*D', 'A*D': 'B*C'}
